In [1]:
%load_ext pretty_jupyter

In [2]:
import patsy as pa
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd

tfd = tfp.distributions

from MakeMyPrior.elicitation_wrapper import expert_model
from MakeMyPrior.training import trainer
from MakeMyPrior.helper_functions import group_obs, Exponential_unconstrained, Normal_unconstrained
from MakeMyPrior.user_config import target_config, target_input
from MakeMyPrior.helper_functions import print_restab, plot_priors_hyp, plot_expert_preds, group_stats, plot_priors_flow
import MakeMyPrior.combine_losses 

from tabulate import tabulate
import pprint
pp = pprint.PrettyPrinter(depth=4)

tf.random.set_seed(2024)

C:\Users\flobo\anaconda3\envs\make-my-prior\Lib\site-packages\bayesflow\trainers.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Background: Case Study
We utilize a Binomial response distribution with a logit-link function for the probability parameter. As accompanying example, we use the Haberman’s survival dataset from the UCI machine learning repository. The dataset contains cases from a study that was
conducted between 1958 and 1970 at the University of Chicago’s Billings Hospital on the survival of patients who had undergone surgery for breast cancer. In the following, we use the detected number of axillary lymph nodes that contain cancer (i.e., (positive) axillary nodes) as numerical predictor $X$ which consists in total of 31 observations ranging between 0 and 59 axillary nodes. The dependent variable $y$ is the number of patients who died within five years out of $T=100$ trials for each observation $i = 1, \ldots ,N$. We consider a simple Binomial regression model with one continuous predictor.
## Data generating model
\begin{align*}
    y_i &\sim \text{Binomial}(T, \theta_i)\\
    \text{logit}(\theta_i) &= \beta_0 + \beta_1x_i\\
    \beta_k &\sim \text{Normal}(\mu_k, \sigma_k) \quad \text{for }k=0,1\\
\end{align*}
The probability parameter $\theta_i$ is predicted by a continuous predictor $x$ with an intercept $\beta_0$ and slope $\beta_1$. We assume normal priors for the regression coefficients, with mean $\mu_k$ and standard deviation $\sigma_k$ for $k=0,1$. Through the logit-link function, the probability $\theta_i$ is mapped to the scale of the linear predictor. The objective is to learn the hyperparameters $\lambda_k=(\mu_k, \sigma_k)$ based on expert knowledge.

# Methodology: Workflow
[//]: # (-.- .tabset)
 
+ General procedure:
    + Draw samples from prior distribution(s) of model parameters 
    + Generate prior predictions according to the data generating model
    + Compute the pre-defined target quantities
    + Compute the elicited statistics of the target quantities
    + Measure the discrepancy between the model-implied and the expert elicited statistics
    + Update the weights of the transformation function of the normalizing flow  


## Method: Hyperparameter Learning
[//]: # (-.- .tabset)

### Method specification
+ PriorSamples($\lambda = (\mu_k, \sigma_k)$):
\begin{align*}
\{\beta_k\}_s &\sim \text{Normal}(\mu_k, \exp\{\sigma_k\})\\
\end{align*}
+ Generator($\beta_k$):
\begin{align*}
\{\text{logit}(\theta_i)\}_s &= \{\beta_k\}_s \times X_i\\
\{y_i\}_s &\sim \text{Binomial}(T, \{\theta_i\}_s)\\
\end{align*}
+ Targets($\{y_i\}_s$):
\begin{align*}
\{y_j\}_s &= \{y_j\}_s \quad \text{ for } j = 0, 5, 10, ,\ldots,30\\
\{R^2\}_s &= \frac{Var(\{\theta_i\}_s)}{Var(\{y_i\}_s)} 
\end{align*}
+ Elicits($\{y_{j}\}_s, \{R^2\}_s$)
\begin{align*}
\text{Quantile-based:} \quad &Q_p^{j} = Q_{p}^{j} \{y_j\}_s \quad \text{ for } j = 0, 5, 10, ,\ldots,30, p = 0.1, \ldots, 0.9\\
\text{Histogram-based:} \quad &\{R^2\}_s =\{R^2\}_s 
\end{align*}

### User specification

#### Setting hyperparameter for the learning algorithm

In [3]:
# -.-|m { input: true, output_error: false, input_fold: show }

# setting of hyperparameter of learning algorithm
user_config = dict(                    
        B = 2**8,                          
        rep = 300,                         
        epochs = 300,                      
        view_ep = 30,
        lr_decay = True,
        lr0 = 0.01, 
        lr_min = 0.0001, 
        loss_dimensions = "m,n:B",   
        loss_discrepancy = "energy", 
        loss_scaling = "unscaled",         
        method = "hyperparameter_learning"  
        )

#### Define the design matrix

In [4]:
# construct design matrix
d = pd.read_csv('C:/Users/flobo/hyp_learn_prior/tests/haberman_prep.csv')
X = tf.constant(d["no_axillary_nodes"], dtype=tf.float32)
x_sd = tf.math.reduce_std(X)
# scale predictor
X_scaled = tf.constant(X, dtype=tf.float32)/x_sd
# select only data points that were selected from expert
dmatrix = tf.gather(X_scaled, [0, 5, 10, 15, 20, 25, 30]) 

pp.pprint(dmatrix)

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([0.       , 0.4075872, 0.8151744, 1.2227615, 1.6303488, 2.037936 ,
       4.238907 ], dtype=float32)>


#### Setup the 'ideal agent'
Define an 'arbitrary' ground truth for the hyperparameter values for method validation.

In [5]:
# true hyperparameter values for ideal_expert
true_values = dict()
true_values["mu"] = [-0.51, 0.26]
true_values["sigma"] = [0.06, 0.04]

# model parameters
parameters_dict = dict()
for i in range(2):
    parameters_dict[f"beta_{i}"] = {
            "family":  Normal_unconstrained(),
            "true": tfd.Normal(true_values["mu"][i], true_values["sigma"][i]),
            "initialization": [tfd.Uniform(0.,1.), tfd.Normal(tf.math.log(0.1), 0.001)]
            }

print("True hyperparameter values:")
pp.pprint(true_values)

True hyperparameter values:
{'mu': [-0.51, 0.26], 'sigma': [0.06, 0.04]}


#### Define the data generating model

In [6]:
# -.-|m { input: true, output_error: false, input_fold: show }

# generative model
class GenerativeModel(tf.Module):
    def __call__(self, 
                 parameters, # obligatory: samples from prior distributions; tf.Tensor
                 dmatrix,    # optional: design matrix; tf.Tensor
                 total_count,       # optiona: total count for Binomial likelihood
                 **kwargs    # obligatory: possibility for further keyword arguments is needed 
                 ):  

        # linear predictor
        theta = tf.expand_dims(parameters[:,:,0], -1) + tf.expand_dims(parameters[:,:,1], -1)*dmatrix
      
        # map linear predictor to theta
        epred = tf.sigmoid(theta)
        
        # define likelihood
        likelihood = tfd.Binomial(
            total_count = total_count, 
            probs = epred[:,:,:,None]
        )
        
        return dict(likelihood = likelihood,      # obligatory: likelihood; callable
                    ypred = None,                # obligatory: prior predictive data
                    epred = epred,                # obligatory: samples from linear predictor
                    theta = theta
                    )

#### Specify the target quantities and the elicitation technique

In [7]:
# specify target quantity, elicitation technique and loss combination
t1 = target_config(target="y_obs", 
                   elicitation = "quantiles", 
                   combine_loss = "by-group", 
                   quantiles_specs = (10, 20, 30, 40, 50, 60, 70, 80, 90))


target_info = target_input(t1)

pp.pprint(target_info)

{'combine_loss': ['by-group'],
 'custom_target_function': [None],
 'elicitation': ['quantiles'],
 'internal_loss': [None],
 'quantiles_specs': [(10, 20, 30, 40, 50, 60, 70, 80, 90)],
 'target': ['y_obs']}


### Simulate from the "ideal" expert

In [8]:
# -.-|m { input: true, output_error: false, input_fold: show }

expert_res_list, prior_pred_res = expert_model(1, user_config["rep"],
                                   parameters_dict, GenerativeModel, target_info,
                                   method = "ideal_expert",
                                   dmatrix = dmatrix,
                                   total_count = 30)

# elicited statistics from the (ideal) expert
expert_res_list.keys()

dict_keys(['y_obs_quant_0'])

#### Expert predictions

In [9]:
# number of patients who died within five years out of 𝑇 = 100 trials
# for 0, 5, 10, 15, 20, 25, 30 axillary nodes
pp.pprint(dict({
    "no.axillary.nodes": [0, 5, 10, 15, 20, 25, 30],
    "no.pat.died": tf.reduce_mean(tf.stack(expert_res_list["y_obs_quant_0"], -1), (0,1)).numpy()
}))

{'no.axillary.nodes': [0, 5, 10, 15, 20, 25, 30],
 'no.pat.died': array([11.218554, 12.015126, 12.821019, 13.578913, 14.263377, 15.052743,
       19.274298], dtype=float32)}


In [10]:
# plot expert predictions
from statsmodels.graphics.boxplots import violinplot
import polars as pl

q_exp = expert_res_list["y_obs_quant_0"]

df = pl.DataFrame( ) 
df = df.with_columns(
    q = np.arange(0.1,1., 0.1)
)
for i,j in zip(range(7),[0, 5, 10, 15, 20, 25, 30]):
    df = df.with_columns(pl.Series(f"{j}", q_exp[i][0,:].numpy())) 
df = df.melt(id_vars = "q", value_vars = [f"{j}" for i,j in zip(range(7), [0, 5, 10, 15, 20, 25, 30])], variable_name = "no.nodes")

df_pd = df.to_pandas()

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

fig, axs = plt.subplots(1,1, layout='constrained', figsize=(6, 3))
[sns.scatterplot(y = j, x = q_exp[j][0,:], ax = axs, zorder = 1) for j in range(7)]
sns.boxplot(x = df_pd["value"], y = df_pd["no.nodes"], color=".99",  linewidth=.75, zorder = 0) 
axs.set_ylabel("number of nodes")
axs.set_xlabel("$Q_p^G$")
axs.set_title("Quantile-based elicitation", loc = "left", pad = 10., fontdict = {'fontsize': 14}) 
plt.show()

NameError: name 'sns' is not defined

### Learn the prior distributions

In [ ]:
res_dict = trainer(expert_res_list, user_config["B"], user_config["rep"],
                   parameters_dict, user_config["method"], GenerativeModel,
                   target_info, user_config, loss_balancing = True, save_vals = ["prior_preds"],
                   dmatrix = dmatrix, total_count = 30)

### Results

#### Loss function

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
_, axs = plt.subplots(1,1, constrained_layout = True, figsize = (6,3))
plt.plot(np.arange(len(res_dict["loss_info"])), res_dict["loss_info"], lw = 3) 
plt.xlabel("epochs")
plt.ylabel(r"$L(\lambda)$")
plt.show()

#### Summary of hyperparameter values

In [ ]:
true_vals = [j for i in zip(true_values["mu"],true_values["sigma"]) for j in i]

tab, avg_res = print_restab(method = "hyperparameter_learning", 
                             num_vars = 4, 
                             res = res_dict["hyperparam_info"], 
                             start = 5, 
                             precision = 3, 
                             true_values = true_vals)
tab

#### Learned prior distributions

In [ ]:
import seaborn as sns

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
_, axs = plt.subplots(1,1, constrained_layout = True, figsize = (6,3))
x = tf.range(-0.8,0.5,0.01)
sns.lineplot(x=x,y=tfd.Normal(avg_res[0], avg_res[1]).prob(x), lw = 3, label = rf"$\beta_0 \sim$ N({avg_res[0]:.2f}, {avg_res[1]:.2f})")
sns.lineplot(x=x,y=tfd.Normal(avg_res[2], avg_res[3]).prob(x), lw = 3, label = rf"$\beta_1 \sim$ N({avg_res[2]:.2f}, {avg_res[3]:.2f})")
sns.lineplot(x=x,y=tfd.Normal(true_vals[0], true_vals[1]).prob(x), linestyle= "dashed", color = "black")
sns.lineplot(x=x,y=tfd.Normal(true_vals[2], true_vals[3]).prob(x), linestyle = "dashed", color = "black")
axs.set_xlabel(r"$\beta_0, \beta_1 \sim$ Normal($\cdot$, $\cdot$)")
axs.set_ylabel("density") 
axs.legend(handlelength = 0.4, fontsize = "medium") 
plt.show()